In [13]:
import numpy as np
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Embedding
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [8]:
# Позитивні рецензії
positive_reviews = [
    "This movie was fantastic. The acting was great and the plot was exciting.",
    "The book was amazing. It kept me hooked from start to finish."
]

# Негативні рецензії
negative_reviews = [
    "The movie was terrible. The acting was awful and the plot was boring.",
    "I didn't enjoy the book. It was slow and predictable."
]

In [9]:
# Мітки класів (1 - позитивна, 0 - негативна)
labels = [1, 1, 0, 0]

# Перетворення тексту на послідовності чисел
tokenizer = Tokenizer()
tokenizer.fit_on_texts(positive_reviews + negative_reviews)
sequences = tokenizer.texts_to_sequences(positive_reviews + negative_reviews)

# Додавання нулів до коротших речень
max_length = max([len(seq) for seq in sequences])
padded_sequences = pad_sequences(sequences, maxlen=max_length, padding='post')

# Кількість унікальних слів
vocab_size = len(tokenizer.word_index) + 1

# Розділення даних на позитивні та негативні
positive_data = padded_sequences[:len(positive_reviews)]
negative_data = padded_sequences[len(positive_reviews):]

In [10]:
# Збудова та навчання моделі для позитивних рецензій
positive_model = Sequential()
positive_model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_length))
positive_model.add(LSTM(units=64, activation='relu'))
positive_model.add(Dense(units=1, activation='sigmoid'))
positive_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
positive_model.fit(positive_data, np.array(labels[:2]), epochs=10, verbose=0)

In [11]:
# Збудова та навчання моделі для негативних рецензій
negative_model = Sequential()
negative_model.add(Embedding(input_dim=vocab_size, output_dim=100, input_length=max_length))
negative_model.add(LSTM(units=64, activation='relu'))
negative_model.add(Dense(units=1, activation='sigmoid'))
negative_model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
negative_model.fit(negative_data, np.array(labels[2:]), epochs=10, verbose=0)

In [12]:
# Оцінка моделей
positive_loss, positive_accuracy = positive_model.evaluate(positive_data, np.array(labels[:2]), verbose=0)
negative_loss, negative_accuracy = negative_model.evaluate(negative_data, np.array(labels[2:]), verbose=0)

print(f'Позитивна модель - Втрати: {positive_loss}, Точність: {positive_accuracy}')
print(f'Негативна модель - Втрати: {negative_loss}, Точність: {negative_accuracy}')

Позитивна модель - Втрати: 0.6036549806594849, Точність: 1.0
Негативна модель - Втрати: 0.6207250356674194, Точність: 1.0
